#### Make sure that you have installed the twitter library
- pip install -i https://pypi.anaconda.org/pypi/simple twitter

In [ ]:
path='/home/drk/kanungo/DNSC6211/week13-twitter/PartB/'

In [ ]:
import twitter

def getAuthData():
    import csv
    with open(path+'authdata.csv', 'r') as f:
        reader = csv.reader(f)
        your_list = list(reader)

    authdata = {}   
    for element in your_list:
        authdata[element[0]] = element[1]

    return authdata

In [ ]:
getAuthData()

In [ ]:
def getTwitterData(searchTerm):
    
    authdata = getAuthData()

    CONSUMER_KEY = authdata['CONSUMER_KEY']
    CONSUMER_SECRET = authdata['CONSUMER_SECRET']
    OAUTH_TOKEN = authdata['OAUTH_TOKEN']
    OAUTH_TOKEN_SECRET = authdata['OAUTH_TOKEN_SECRET']
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN,
                               OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY,
                               CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    
    # Nothing to see by displaying twitter_api except that it's now a defined variable
    print (twitter_api)
    
    searchCount = 1800 # The number of tweets you want ... you never get that !
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets
    search_results = twitter_api.search.tweets(q=searchTerm, count=searchCount)
    statuses = search_results['statuses']
    
    # Iterate through 5 more batches of results by following the cursor
    
    for _ in range(5):
        print ("Length of statuses", len(statuses))
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            print ("That's all folks for ...", searchTerm)
            break
        myKVs = [ kv.split('=') for kv in next_results[1:].split("&") ]
        myStrippedKVs = [ [str(pp[0]), str(pp[1])] for pp in myKVs]
        kwargs = dict(myStrippedKVs)    
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
    
    status_texts = [ status['text'] for status in statuses ]
    return status_texts

In [ ]:
# This is new ...
# Get and save twitter data

tweetTexts = getTwitterData('@Delta')
with open(path+'tw_delta.txt','w') as f:
    f.writelines(tweetTexts)
tweetTexts = getTwitterData('@United')
with open(path+'tw_united.txt','w') as f:
    f.writelines(tweetTexts)
tweetTexts = getTwitterData('@USAirways')
with open(path+'tw_usair.txt','w') as f:
    f.writelines(tweetTexts)
tweetTexts = getTwitterData('@Southwest')
with open(path+'tw_southwest.txt','w') as f:
    f.writelines(tweetTexts)
tweetTexts = getTwitterData('@American')
with open(path+'tw_american.txt','w') as f:
    f.writelines(tweetTexts)
tweetTexts = getTwitterData('@JetBlue')
with open(path+'tw_jetblue.txt','w') as f:
    f.writelines(tweetTexts)


In [ ]:
# Get negative and positive word lists
def getWordLists():
    negative = [line.strip() for line in open(path+'neg.txt')]
    negative = [n for n in negative if n != [] ]
    negative = [n for n in negative if n != '' ]
    negative = [n for n in negative if n[0] != ';']
    
    positive = [line.strip() for line in open(path+'pos.txt')]
    positive = [n for n in positive if n != [] ]
    positive = [n for n in positive if n != '' ]
    positive = [n for n in positive if n[0] != ';']

    return negative, positive

In [ ]:
negative , positive= getWordLists()
#print (negative)

In [ ]:
# Data cleaning + Sentiment scores

def remove_punctuation(s):
    punctuation = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    s_sans_punct = ""
    for letter in s:
        if (letter not in punctuation) and (letter in "abcdefghijklmnopqrstuvwxyz "):
            s_sans_punct += letter
    return s_sans_punct

def getLowerCaseText(status_texts):
    lowered_texts = []
    for texts in status_texts:
        try: 
            mytext = str(texts.lower())
            lowered_texts.append(mytext)
        except:
            pass
    return lowered_texts

def getCleanedTweets(lowered_texts):
    cleanedTweets = []
    for text in lowered_texts:
        wordlist = str(text).split()
        wordlist_nopun = [ str(remove_punctuation(for_a_word)) for for_a_word in wordlist ]
        cleanedTweets.append(wordlist_nopun)
    return cleanedTweets

def GetSentimentScores(cleanedTweets, negative, positive):
    freqList = []
    for eachTweet in cleanedTweets:
        posScore = len(set(eachTweet) & set(positive))
        negScore = len(set(eachTweet) & set(negative))
        freqList.append(posScore-negScore)
    return freqList

In [ ]:
# Plotting routines

In [ ]:
# Plotting routines

def plotHist(freqList):
    import matplotlib.pyplot as plt
    plt.hist(freqList)

def getTwitterScore(freqList):
    veryNegative = len([x for x in freqList if x<=-2])
    veryPositive = len([x for x in freqList if x>=+2])
    twitterScore = 100 * (float (veryPositive) / (veryNegative + veryPositive))
    return round(twitterScore, 2)

In [ ]:
# Get airline data
# This should work --- but is not working as of today 
# I have created an alternative function below - manual data entry ... ugh!!
def getAirlineData():
    from bs4 import BeautifulSoup as bs  
    import requests
    url = 'https://www.theacsi.org/?option=com_content&view=article&id=147&catid=14&Itemid=212&i=Airlines'
    r = requests.get(url)
    airlinesSatPage = r.text       
    soup = bs(airlinesSatPage, "html.parser")

    allRows = soup.findAll('tr',{'class':'Company'})
    tableData = []
    for row in allRows:
        eachRow = []
        cells = row.findAll('td')
        for cell in cells:
            eachRow.append(str(cell.text.strip()))
        tableData.append(eachRow)

    airline = [ x[0] for x in tableData ]
    score   = [ x[21] for x in tableData ]
    airlineDict = { 'airlines': airline,
                    'score'   : score
                  }
    
    return airlineDict  

In [ ]:
def getAirlineDataAlt():
    airlineDict = {}
    airlineDict = {'JetBlue':80, 
                   'Southwest':80,
                   'American':72,
                   'Delta':71,
                   'United':68
                  }
    return airlineDict

In [ ]:
airlineScores = getAirlineDataAlt()

In [ ]:
def getScore(fileName):
    fn = path+fileName+'.txt'
    with open(fn,'r') as f:
        tweetTexts = f.readlines()
    lowered_texts = getLowerCaseText(tweetTexts)
    cleanedTweets = getCleanedTweets(lowered_texts)
    freqList = GetSentimentScores(cleanedTweets, negative, positive)
    score = getTwitterScore(freqList)
    return(score)

In [ ]:
scores = {}

scores = {'American': getScore('tw_american'),
          'Delta': getScore('tw_delta'),
          'JetBlue':  getScore('tw_jetblue'),
          'Southwest': getScore('tw_southwest'),
          'United': getScore('tw_united'),
          'USAirways': getScore('tw_usair')
         }

print(scores)

In [ ]:
import pandas as pd
twdf = pd.DataFrame.from_dict(scores, orient='index')
twdf.columns = ['twitterScore']
print(twdf)

In [ ]:
import pandas as pd
airdf = pd.DataFrame.from_dict(airlineScores, orient='index')
airdf.columns = ['airlineScore']
print(airdf)

In [ ]:
set(["a","b","c"]) & set(["a","d","c"])


In [ ]:
result = pd.concat([twdf, airdf], axis=1, join='inner')

In [ ]:
print(result)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.scatter(result['twitterScore'], result['airlineScore'])